<a href="https://colab.research.google.com/github/sanbgos/Se-alesysistemas/blob/main/Parcial1_SyS_2025_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

 -- Para recibir crédito total por sus respuestas, estas deben estar claramente   justificadas e ilustrar sus procedimientos y razonamientos (paso a paso) de forma concreta, clara y completa.

-- El parcial debe ser enviado al correo electrónico amalvarezme@unal.edu.co antes de las 23:59 del 16 de mayo de 2025, vía link de GitHub, con com
ponentes teóricas de solución a mano en formato pdf y componentes de simulación en un cuaderno de Python. Si el orreo unal o GitHub presentan inconsistencias, enviar los archivos como adjunto en .zip.

-- Loscódigos deben estar debidamente comentados y discuti
dos en celdas de texto (markdown). Códigos no comentados ni discutidos, no serán contabilizados en la nota final.

-- Incluir en el asunto del correo de env´ ıo del parcial: Parcial 1 SyS 2025-1: Nombre completo.

# Pregunta 1
Se tiene un microprocesador de 5 bits con entrada análoga de -3.3 a 5[v]. Diseñe el sistema de acondicionamiento y digitalización para la señal: $x(t) = 20\sin(7t−π/2)−3\cos(5t)+2\cos(10t)$. Presente las simulaciones y gráficas de los pro
cedimientos más representativos en un cuaderno de Python, incluyendo al menos dos períodos de la señal estudiada.

Graficamos la señal analógica original

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Período fundamental de la señal compuesta
T_0 = 2*np.pi # Periodo fundamental o total

# Amplitudes
A_1 = 20
A_2 = 3
A_3 = 2

# Frecuencias angulares
w1 = 7     # rad/s
w2 = 5     # rad/s
w3 = 10    # rad/s

# Tiempo de simulación (2 períodos fundamentales)
ti = -T_0
tf = T_0

# Frecuencia de muestreo: al menos el doble de la máxima frecuencia (Nyquist)
f_max = w3 / (2 * np.pi)  # frecuencia en Hz
Fs = 10 * f_max           # muestreo 10 veces mayor para buena visualización
Ts = 1 / Fs

# Vector de tiempo
tv = np.arange(ti, tf, Ts)

# Construcción de la señal
xv = A_1 * np.sin(w1 * tv-np.pi/2) - A_2 * np.cos(w2 * tv) + A_3 * np.cos(w3 * tv)

# Graficar señal continua
plt.plot(tv, xv, 'b')
plt.xlabel('Tiempo [s]')
plt.ylabel('x(t)')
plt.title('Señal compuesta muestreada (2 períodos)')
plt.grid(True)
plt.show()

In [ ]:
# Definir la función my_ceropen (cero y pendiente)
def my_ceropen(x, ymin=0, ymax=200):
  xmax = max(x)
  xmin = min(x)
  m = (ymax - ymin) / (xmax - xmin)
  c = ymin - m * xmin
  yv = m * x + c
  return yv

# Redefinir "y" como se definió en la parte anterior
y = 20 * np.sin(7 * tv-np.pi/2) - 3 * np.cos(5 * tv) + 2 * np.cos(10 * tv)

  #cero y pendiente
ycs = my_ceropen(y, ymin=-3.3,ymax=3.3)


#número de bits y vector de cuantización
nbits = 5
rmin = -3.3
rmax = 5
ve = np.linspace(rmin,rmax,2**nbits) # (rmax-rmin)/(2^#bits - 1) -> si se quiere incluir rmin y rmax
from scipy.spatial.distance import cdist


# Función de cuantización
def my_cuantizador(yn, vq):
  dn = cdist(np.array(yn).reshape(-1, 1), vq.reshape(-1, 1))
  ind = np.argmin(dn, axis=1)
  return vq[ind]

# Cuantizar la señal
yq = np.zeros_like(ycs)
for i in range(len(ycs)):
  yq[i] = my_cuantizador(ycs[i], ve)

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(tv, ycs, c='black', label='ycs') # Señal original
plt.stem(tv, ycs, label='ycs_dis') # Señal discretizada
plt.plot(tv, yq, c='y', marker='o', label='ydig', linewidth=2)  # Señal cuantificada
# Mostrar niveles de cuantificación
for v in ve:
  plt.plot(tv,v*np.ones(len(tv)))
plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.title('Señal Original, Discretizada y Cuantificada')
plt.show()


# Pregunta 2
¿Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital con frecuencia de muestreo de 5kHz, aplicado a la señal $x(t) = 3\cos(1000πt) + 5\sin(2000πt) + 10\cos(11000πt)?$. Realizar la simulación del proceso de discretización. En caso de que la discretización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros de la señal
Fs = 5000  # Frecuencia de muestreo (Hz)
t = np.linspace(0, 0.01, 1000)  # Tiempo continuo (10 ms)

# Frecuencias angulares continuas
w1 = 1000 * np.pi
w2 = 2000 * np.pi
w3 = 11000 * np.pi

# Señal continua
x_t = 3 * np.cos(w1 * t) + 5 * np.sin(w2 * t) + 10 * np.cos(w3 * t)

# Señal discretizada
N = 50  # Número de muestras
n = np.arange(N)

# Frecuencias angulares normalizadas (Omega = w / Fs)
Omega1 = w1 / Fs
Omega2 = w2 / Fs
Omega3 = w3 / Fs

# Aliasing: llevar Omega3 al intervalo [-pi, pi]
if Omega3 > np.pi:
    Omega3_alias = Omega3 - 2*np.pi
else:
    Omega3_alias = Omega3

# Mostrar frecuencias normalizadas antes y después de aliasing
print(f"Ω1 = {Omega1:.2f} rad/muestra")
print(f"Ω2 = {Omega2:.2f} rad/muestra")
print(f"Ω3 antes de aliasing = {Omega3:.2f} rad/muestra")
print(f"Ω3 después de aliasing = {Omega3_alias:.2f} rad/muestra")

# Señal discretizada después de aliasing
x_n = (13) * np.cos(Omega1 * n) + 5 * np.sin(Omega2 * n)

# Graficar señal continua y discretizada
plt.figure(figsize=(14, 6))

# Señal continua
plt.subplot(2, 1, 1)
plt.plot(t*1000, x_t, label='Señal continua $x(t)$', color='orange')
plt.title('Señal continua original')
plt.xlabel('Tiempo (ms)')
plt.ylabel('Amplitud')
plt.grid(True)
plt.legend()

# Señal discretizada
plt.subplot(2, 1, 2)
plt.stem(n, x_n, basefmt=" ", label='Señal discretizada $x[n]$', linefmt='orange', markerfmt='o')
plt.title('Señal discretizada (con de aliasing)')
plt.xlabel('Muestra n')
plt.ylabel('Amplitud')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros de la señal
A1, A2, A3 = 3, 5, 10
w1 = 1000 * np.pi   # 500 Hz
w2 = 2000 * np.pi   # 1000 Hz
w3 = 11000 * np.pi  # 5500 Hz

# Intervalo de tiempo corto para visualización
t_12k = np.arange(0, 0.005, 1/12000)

# Señal muestreada a 12 kHz (sin componente de 5500 Hz por filtrado)
x_12k = A1 * np.cos(w1 * t_12k) + A2 * np.sin(w2 * t_12k)

# Graficar ambas señales
plt.figure(figsize=(14, 6))

plt.subplot(2,1,2)
plt.stem(t_12k, x_12k, linefmt='b-', markerfmt='bo', basefmt='k-')
plt.title('Señal muestreada a 12 kHz (SIN aliasing, con filtrado anti-aliasing)')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')

plt.tight_layout()
plt.show()

# Pregunta 3
La distancia media entre dos señales $x1(t),x2(t) ∈ R,C,$ se puede expresar a partir de la potencia media:
$$
d(x_{1},x_{2})=\bar{P}_{x_{1}-x_{2}}=\displaystyle \lim_{T \to \infty }\frac{1}{T}\int_{T}^{}\left | x_{1}(t)-x_{2}(t)\right |^{2}dt
$$
Sean $x_{1}(t)$ y $x_{2}(t)$:
$$
x_{1}(t)=A\,cos(\omega _{0}t),\quad\omega _{0}=\frac{2\pi }{T},\quad T,A \in\mathbb{R}^{+}
$$

$$
x_{2}(t)=\begin{cases}
   1         & \text{si }\quad 0\leq t < \frac{T}{4} \\
  -1         & \text{si }\quad  \frac{T}{4}\leq t < \frac{3T}{4} \\
   1   & \text{si }\quad \frac{3T}{4}\leq t < T
\end{cases}
$$

¿Cuál es la distancia media entre las señales?. Corrobore sus
 desarrollos con Sympy.

Analizando las dos señales, nos damos cuenta que tenemos una señal a trozos ($x_{2}(t)$), por lo tanto, dividimos los intervalos, donde optenemos 3 integrales:
$$
\bar{P}_{x_{1}-x_{2}}=\displaystyle \lim_{T \to \infty }\frac{1}{T}\left [ \int_{0}^{T/4} \left ( A\,cos(w_{0}t)-1 \right )^{2}dt\;+\;\int_{T/4}^{3T/4} \left ( A\,cos(w_{0}t)+1 \right )^{2}dt\;+\;\int_{3T/4}^{T} \left ( A\,cos(w_{0}t)-1 \right )^{2}dt\right ]
$$

Asi que procedemos a hallar las integrales.

Resolviendo la primera integral en el intervalo de $0\leq t < T/4$
$$
\int_{0}^{T/4} \left ( A\,cos(w_{0}t)-1 \right )^{2}dt
$$

In [ ]:
import sympy as sp

# Definir las variables simbólica
t, T, A = sp.symbols('t T A')
w0=(2*sp.pi)/T

# Definir la función a integrar (puedes cambiarla aquí)
f1 = (A*sp.cos(w0*t)-1)**2

# Definir los límites de integración
a = 0       #Limite inferior
b = T/4     #Limite superior

# Calcular la integral definida
integral_definida1 = sp.integrate(f1, (t, a, b))

# Mostrar resultado
print("La integral definida de ", f1, "desde", a, "hasta", b, "es:")
#print(integral_definida)
integral_definida1


Resolviendo la segunda integral en el intervalo de $T/4\leq t < 3T/4$
$$
\int_{T/4}^{3T/4} \left ( A\,cos(w_{0}t)+1 \right )^{2}dt
$$

In [ ]:
import sympy as sp

# Definir las variables simbólica
t, T, A = sp.symbols('t T A')
w0=(2*sp.pi)/T

# Definir la función a integrar (puedes cambiarla aquí)
f2 = (A*sp.cos(w0*t)+1)**2

# Definir los límites de integración
a = T/4      #Limite inferior
b = 3*T/4     #Limite superior

# Calcular la integral definida
integral_definida2 = sp.integrate(f2, (t, a, b))

# Mostrar resultado
print("La integral definida de ", f2, "desde", a, "hasta", b, "es:")
#print(integral_definida)
integral_definida2


Resolviendo la tercera integral en el intervalo de $3T/4\leq t < T$
$$
\int_{3T/4}^{T} \left ( A\,cos(w_{0}t)-1 \right )^{2}dt
$$

In [ ]:
import sympy as sp

# Definir las variables simbólica
t, T, A = sp.symbols('t T A')
w0=(2*sp.pi)/T

# Definir la función a integrar (puedes cambiarla aquí)
f3 = (A*sp.cos(w0*t)-1)**2

# Definir los límites de integración
a = 3*T/4       #Limite inferior
b = T     #Limite superior

# Calcular la integral definida
integral_definida3 = sp.integrate(f3, (t, a, b))

# Mostrar resultado
print("La integral definida de ", f3, "desde", a, "hasta", b, "es:")
#print(integral_definida)
integral_definida3

In [ ]:
R= integral_definida1+integral_definida2+integral_definida3
print("Despues de resolver la integral dividimos por T")
d=R/T
X = d.simplify()
X


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
W0=2
T0= 2*np.pi/W0
N=1000
t= np.linspace(0, T0, N)
x1 = np.cos(W0*t)

T=T0

def x2(t_local, T):
    """Define x2 en un solo período entre 0 y T"""
    if 0 <= t_local < T/4:
        return 1
    elif T/4 <= t_local < 3*T/4:
        return -1
    elif 3*T/4 <= t_local < T:
        return 1
    else:
        return 0

# Vectorizar x2 para varios períodos
x2 = np.array([x2(ti % T, T) for ti in t])

# Graficar
plt.figure(figsize=(12, 6))

# Señal x1(t)
plt.subplot(2, 1, 1)
plt.plot(t, x1, label=r'$x_1(t) = \cos(\omega_0 t)$')
plt.title("Señal x₁(t) = cos(ω₀t)")
plt.grid(True)
plt.legend()

# Señal x2(t)

plt.subplot(2, 1, 2)
plt.plot(t, x2, color='orange', label=r'$x_2(t)$')
plt.title("Señal x₂(t): señal periódica por tramos")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


# Pregunta 3
Sea $x′′(t)$ la segunda derivada de la señal $x(t)$, donde $t$ $∈$ $[ti, tf]$. Demuestre que los coeficientes de la serie exponencial de Fourier se pueden calcular según:
$$
c_{n}=\frac{1}{(ti-tf)n^{2}w_{o}^{2}}\int_{t_i}^{t_f}x^{''}(t)e^{-jnw_{o}t}dt;\quad n\in\mathbb{Z}
$$
¿Cómo se pueden calcular los coeficientes an y bn desde $x′′(t)$ en la serie trigonométrica de Fourier?.

Encuentre el espectro de Fourier, su magnitud, fase, parte real, parte imaginaria y el error relativo de reconstrucción para $n∈{0,±1,±2,±3,±4,±5}$, a partir de $x′′(t)$ para la señal $x(t)$ en la Figura 1 . Compruebe el espectro obtenido con la estimación a partir de $x(t)$ y presente las respectivas simulaciones sobre Python.

In [ ]:
# Importación de librerías para gráficos y manipulación numérica
from ipywidgets import interact,IntSlider
import numpy as np
import matplotlib.pyplot as plt

# Parámetros para definir la escala y forma de la señal
A=1        #Amplitud de la señal
d2=2       #Extremo derecho del trapecio
d1=1       #Límite de la parte plana del trapecio

#Generar señal parametrizada en función del tiempo
T = 2*d2       # Periodo total de la señal
Fo = 1/T       # Frecuencia fundamental de la señal
Fs = 1000*Fo   # Frecuencia de muestreo muy alta para buena resolución
tv = np.arange(-T/2,T/2,1/Fs) # Vector de tiempo para graficar la señal
Nm = len(tv)   # Cantidad de puntos evaluados

# Definición de una función que genera una señal trapezoidal en base a cuatro puntos clave
def funcion_trapecio(x, a, b, c, d):
    """
Parámetros:
x: vector de tiempo
a: inicio de rampa ascendente
b: inicio de la meseta
c: final de la meseta
d: final de rampa descendente
    """
    # Implementación por tramos con funciones máximo y mínimo
    return np.maximum(0, np.minimum((x - a) / (b - a), np.minimum(1, (d - x) / (d - c))))

# Se genera la señal evaluando en el vector de tiempo y usando los puntos establecidos
x = -funcion_trapecio(tv, -d2, -d1, d1, d2)

#graficar señal original
plt.plot(tv,x,'b',linewidth=4)
plt.grid()
plt.xlabel("t[s]",fontsize = 14)
plt.ylabel("x(t)",fontsize = 14)
plt.show()

In [ ]:
N = 5  # Cantidad de armónicos positivos y negativos a considerar
wo = 2 * np.pi / T  # Frecuencia angular fundamental de la señal periódica

# Generar las funciones base complejas para los armónicos de Fourier.
# phin será una matriz que almacenará las exponenciales complejas correspondientes a cada armónico.
phin = np.zeros((Nm, 2*N+1), np.complex128)  # Inicializa matriz de ceros de tipo complejo

# Se recorre desde -N hasta N para calcular el valor de cada exponencial compleja
for n in range(-N, N+1, 1):
    phin[:, n+N] = np.exp(1j * n * wo * tv)  # Cálculo de la base exponencial para cada armónico

# Calcular manualmente los coeficientes Cn de la serie de Fourier para la señal generada
cn = np.zeros(2*N+1, np.complex128)  # Vector para guardar los coeficientes complejos

nv = np.linspace(-N, N, 2*N+1)  # Vector con los valores de los armónicos (índice n)

# Fórmula teórica de los coeficientes Cn de Fourier para un trapecio simétrico
# Se obtiene aplicando la integral de Fourier a la forma trapezoidal definida antes
Cn = (-2*(np.cos(nv*np.pi) - np.cos(nv*np.pi*0.5))) / (np.pi * 2 * nv * 2)

# Se ajusta el valor de la componente DC (n=0), que no puede calcularse con la fórmula anterior
Cn[N] = 3/4  # Valor medio de la señal (componente de frecuencia cero)



#graficar espectro
fig = plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.stem(nv, np.real(Cn), 'r')
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$Re\{C_n\}$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,2)
plt.stem(nv, np.imag(Cn), 'r')
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$Im\{C_n\}$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,3)
plt.stem(nv, abs(Cn), 'r')#magnitud del espectro
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$|C_n|$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,4)
plt.stem(nv, np.angle(Cn), 'r')#fase del espectro
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$\langle C_n$',fontsize = 14)
plt.axis('tight')
plt.grid()

fig.tight_layout()

In [ ]:
Px = 2/3 # Potencia de la señal original

# La función pltest, ayuda a la reconstrucción y grafico de la señal
def pltest(Na=1):
    ind = np.arange(N-Na,N+Na+1)
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    #señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] ) # dot calcula el producto entre vectores
    plt.plot(tv,xe,color='b',label='$x_e(t)$') # señal estimada o filtrada
    plt.plot(tv,x,color='r',label='x(t)') #señal original
    plt.title('$E_r$=%.2f[%%]' % (100*er),fontsize=16)
    plt.xlabel('t[s]')
    plt.ylabel('x(t)')
    plt.grid()
    plt.legend()
    plt.show()

    return
interact(pltest,Na=(1,N,1))